In [ ]:
import collections
import numpy
import os
import sklearn.model_selection

from classifier import classification_data

cfg = {
    "fold": 5,
    "n_labels": 5,
    "path_base": "dataset",
    "path_out": "out",
    "test_size": 0.2,
    "train_size": 0.8,
    "random_state": 1
}

In [ ]:
surf = numpy.loadtxt(os.path.join(cfg["path_base"], "surf64.txt"))
n_samples, n_features = surf.shape
x_surf, y_surf = surf[0:, 0:n_features - 1], surf[:, n_features - 1]

kf = sklearn.model_selection.KFold(n_splits=cfg["fold"], shuffle=True, random_state=cfg["random_state"])
l = list([])
for (train_index, test_index) in kf.split(x_surf):
    l = l + test_index.tolist()
print(len(list(set(l))))
print(collections.Counter(sorted(l)))

In [ ]:
mobilenet = numpy.load(os.path.join(cfg["path_base"], "cnn", "horizontal+patch=3+mobilenetv2.npz"))
dataset = "mobilenetv2"
orientation = "horizontal"
n_patch = 3
x = mobilenet["features"]
y = mobilenet["labels"]
n_samples = x.shape[0]
n_features = x.shape[1]
x_normalized = sklearn.preprocessing.StandardScaler().fit_transform(x)

In [ ]:
for pca in (128, 256, 512, 1024, -1):
    x = x_normalized
    if pca > 0:
        x = sklearn.decomposition.PCA(n_components=pca).fit_transform(x)

    if pca == -1:
        pca = n_features

    classification_data(cfg, dataset, list(kf.split(x_surf)), pca, n_samples, x, y, n_patch=n_patch, orientation=orientation)